In [1]:
%matplotlib notebook
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as td
import torchvision as tv
from PIL import Image
import matplotlib.pyplot as plt
import vocModel.nntools as nt
import vocData as voc
import vocModel
from torch.utils.data import DataLoader
import cv2
import time

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
device = 'cpu'

cuda


In [3]:
dataset_root_dir = "/datasets/ee285f-public/PascalVOC2012/"

In [4]:
train_set = voc.VOCDetection_Yolo(dataset_root_dir,  image_set = 'train')
val_set = voc.VOCDetection_Yolo(dataset_root_dir, image_set = 'val')
print (len(train_set), len(val_set))


5717 5823


In [5]:
net = vocModel.YoloNet(7, 2, 20, 5,0.5)
net = net.to(device)
lr = 1e-4
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

In [6]:
train_loader = DataLoader(train_set,batch_size=3,shuffle=False,num_workers=0)
train_iter = iter(train_loader)
for i in range(1):
    img,target = next(train_iter)
    img,target = img.to(device), target.to(device)

    output = net.forward(img)
#     loss = net.criterion(output, target)
#     print (loss)    
#     loss.backward()
#     optimizer.step()
    

In [18]:
loss = my_forward(output, target)
print (loss)

torch.Size([7, 30])
tensor(20.4206, grad_fn=<AddBackward0>)


In [20]:
def my_forward(pred, target):
 

#     "referce : https://github.com/thtrieu/darkflow/tree/master/darkflow/net (Tensor Flow)"


    loss=0

    lamb_class_prob = 1
    lamb_obj_conf = 1
    lamb_noobj_conf = 0.5
    lamb_coord = 5
    
    batch_size = y_pred.size(0)

    
    
#     loss = 0
#     n = pred.size()[0]
#     object_mask = target[:,:,:,4] > 0
#     object_mask = object_mask.unsqueeze(-1).expand_as(target)
#     # 1. Classification loss   
#     class_pred = pred[object_mask].view(-1,30)
#     print (class_pred.size())
#     class_pred = class_pred[:,20:]
#     class_target = target[object_mask].view(-1,30)
#     class_target = class_target[:,20:]
    
    
    
#     class_loss = F.mse_loss(class_pred,class_target,reduction='sum')
    
#     # 2. Localization loss
#     coords_pred = pred[]
    
    
    return loss
    

In [97]:
a = torch.rand((3,3))
print (a)
b = torch.from_numpy(np.array([[1,1,1],[0,0,0],[1,1,1]]))
print (b)

tensor([[0.3114, 0.0603, 0.3071],
        [0.2400, 0.2074, 0.1113],
        [0.5634, 0.0395, 0.3035]])
tensor([[1, 1, 1],
        [0, 0, 0],
        [1, 1, 1]])


In [98]:
c = a[b]

In [99]:
print (c.size()
      )

torch.Size([3, 3, 3])


In [23]:
a = torch.randn(3, 4, 5)
b = a.permute(1, 2, 0)
c = b.contiguous()
d = a.contiguous()
# a has "standard layout" (also known as C layout in numpy) descending strides, and no memory gaps (stride(i-1) == size(i)*stride(i))
print (a.size(), a.shape, a.stride(), a.data_ptr())
print (a)
# b has same storage as a (data_ptr), but has the strides and sizes swapped around
print (b.size(), b.shape, b.stride(), b.data_ptr())
print (b)
# c is in new storage, where it has been arranged in standard layout (which is "contiguous")
print (c.size(), c.shape, c.stride(), c.data_ptr())
print (c)
# d is exactly as a, as a was contiguous all along
print (d.size(), d.shape, d.stride(), d.data_ptr())

torch.Size([3, 4, 5]) torch.Size([3, 4, 5]) (20, 5, 1) 94569426107440
tensor([[[-0.0927, -0.0730,  0.3672,  1.3897, -0.5202],
         [ 0.3496,  1.9922, -0.1502,  0.8792,  2.1860],
         [ 0.8653,  0.9986,  0.6206, -0.3740,  0.6920],
         [ 0.4099, -1.8327, -0.2812,  0.2164, -0.1108]],

        [[ 0.2095,  0.6548, -0.9549, -1.3656, -0.5294],
         [ 1.1057,  1.8712,  0.9089, -0.5541,  0.5597],
         [ 1.2444,  0.6688,  1.0857,  2.3752, -0.3824],
         [ 0.2090, -1.5950, -1.3451, -0.5951,  0.3794]],

        [[-1.8272,  0.5864, -0.6422,  2.5283, -1.8432],
         [-0.2214,  1.1106,  1.5708, -0.6046, -0.6657],
         [ 0.2525,  0.6406, -0.6576, -0.1579, -1.3209],
         [ 1.5862,  0.1400,  0.0511, -0.6695, -1.6985]]])
torch.Size([4, 5, 3]) torch.Size([4, 5, 3]) (5, 1, 20) 94569426107440
tensor([[[-0.0927,  0.2095, -1.8272],
         [-0.0730,  0.6548,  0.5864],
         [ 0.3672, -0.9549, -0.6422],
         [ 1.3897, -1.3656,  2.5283],
         [-0.5202, -0.5294, -1

In [ ]:
def forward(pred_tensor,target_tensor):

    N = pred_tensor.size()[0]
    coo_mask = target_tensor[:,:,:,4] > 0
    noo_mask = target_tensor[:,:,:,4] == 0
    coo_mask = coo_mask.unsqueeze(-1).expand_as(target_tensor)
    noo_mask = noo_mask.unsqueeze(-1).expand_as(target_tensor)

    coo_pred = pred_tensor[coo_mask].view(-1,30)
    box_pred = coo_pred[:,:10].contiguous().view(-1,5) #box[x1,y1,w1,h1,c1]
    class_pred = coo_pred[:,10:]                       #[x2,y2,w2,h2,c2]

    coo_target = target_tensor[coo_mask].view(-1,30)
    box_target = coo_target[:,:10].contiguous().view(-1,5)
    class_target = coo_target[:,10:]

    # compute not contain obj loss
    noo_pred = pred_tensor[noo_mask].view(-1,30)
    noo_target = target_tensor[noo_mask].view(-1,30)
    noo_pred_mask = torch.cuda.ByteTensor(noo_pred.size())
    noo_pred_mask.zero_()
    noo_pred_mask[:,4]=1;noo_pred_mask[:,9]=1
    noo_pred_c = noo_pred[noo_pred_mask] #noo pred只需要计算 c 的损失 size[-1,2]
    noo_target_c = noo_target[noo_pred_mask]
    nooobj_loss = F.mse_loss(noo_pred_c,noo_target_c,size_average=False)

    #compute contain obj loss
    coo_response_mask = torch.cuda.ByteTensor(box_target.size())
    coo_response_mask.zero_()
    coo_not_response_mask = torch.cuda.ByteTensor(box_target.size())
    coo_not_response_mask.zero_()
    box_target_iou = torch.zeros(box_target.size()).cuda()
    for i in range(0,box_target.size()[0],2): #choose the best iou box
        box1 = box_pred[i:i+2]
        box1_xyxy = torch.FloatTensor(box1.size())
        box1_xyxy[:,:2] = box1[:,:2]/14. -0.5*box1[:,2:4]
        box1_xyxy[:,2:4] = box1[:,:2]/14. +0.5*box1[:,2:4]
        box2 = box_target[i].view(-1,5)
        box2_xyxy = torch.FloatTensor(box2.size())
        box2_xyxy[:,:2] = box2[:,:2]/14. -0.5*box2[:,2:4]
        box2_xyxy[:,2:4] = box2[:,:2]/14. +0.5*box2[:,2:4]
        iou = self.compute_iou(box1_xyxy[:,:4],box2_xyxy[:,:4]) #[2,1]
        max_iou,max_index = iou.max(0)
        max_index = max_index.data.cuda()

        coo_response_mask[i+max_index]=1
        coo_not_response_mask[i+1-max_index]=1

        #####
        # we want the confidence score to equal the
        # intersection over union (IOU) between the predicted box
        # and the ground truth
        #####
        box_target_iou[i+max_index,torch.LongTensor([4]).cuda()] = (max_iou).data.cuda()
    box_target_iou = box_target_iou.cuda()
    #1.response loss
    box_pred_response = box_pred[coo_response_mask].view(-1,5)
    box_target_response_iou = box_target_iou[coo_response_mask].view(-1,5)
    box_target_response = box_target[coo_response_mask].view(-1,5)
    contain_loss = F.mse_loss(box_pred_response[:,4],box_target_response_iou[:,4],size_average=False)
    loc_loss = F.mse_loss(box_pred_response[:,:2],box_target_response[:,:2],size_average=False) +\
    F.mse_loss(torch.sqrt(box_pred_response[:,2:4]),torch.sqrt(box_target_response[:,2:4]),size_average=False)
    #2.not response loss
    box_pred_not_response = box_pred[coo_not_response_mask].view(-1,5)
    box_target_not_response = box_target[coo_not_response_mask].view(-1,5)
    box_target_not_response[:,4]= 0
    #not_contain_loss = F.mse_loss(box_pred_response[:,4],box_target_response[:,4],size_average=False)

    #I believe this bug is simply a typo
    not_contain_loss = F.mse_loss(box_pred_not_response[:,4], box_target_not_response[:,4],size_average=False)

    #3.class loss
    class_loss = F.mse_loss(class_pred,class_target,size_average=False)
    
    return (self.l_coord*loc_loss + 2*contain_loss + not_contain_loss + self.l_noobj*nooobj_loss + class_loss)/N
                 

In [9]:
def intersect(box_a, box_b):
    A = box_a.size(0)
    B = box_b.size(0)
    
    max_xy = torch.min(box_a[:, 2:].unsqueeze(1).expand(A, B, 2),
                       box_b[:, 2:].unsqueeze(0).expand(A, B, 2))
    min_xy = torch.max(box_a[:, :2].unsqueeze(1).expand(A, B, 2),
                       box_b[:, :2].unsqueeze(0).expand(A, B, 2))
    
    inter = torch.clamp((max_xy - min_xy), min=0)
    return inter[:, :, 0] * inter[:, :, 1]

def cal_iou(box_a, box_b):
    inter = intersect(box_a, box_b)
    area_a = ((box_a[:, 2]-box_a[:, 0]) *
              (box_a[:, 3]-box_a[:, 1])).unsqueeze(1).expand_as(inter)  # [A,B]
    area_b = ((box_b[:, 2]-box_b[:, 0]) *
              (box_b[:, 3]-box_b[:, 1])).unsqueeze(0).expand_as(inter)  # [A,B]
    union = area_a + area_b - inter
    return inter / union  # [A,B]
